In [1]:
import gurobipy as gp
from gurobipy import GRB, quicksum
import pandas as pd

In [2]:
df = pd.read_excel('Railway services-2024.xlsx', index_col='Trip')[:200]
df['P_c'] = df['Demand(μ)']
df['L_c'] = 300 
df.loc[df['Line'] == 400, 'L_c'] = 200

display(df)

/var/folders/4g/c51vpg2j0nj65hpt4ffnh7xm0000gn/T/ipykernel_38927/3127984627.py:1: FutureWarning: The behavior of obj[i:j] with a float-dtype index is deprecated. In a future version, this will be treated as positional instead of label-based. For label-based slicing, use obj.loc[i:j] instead
  df = pd.read_excel('Railway services-2024.xlsx', index_col='Trip')[:200]


,Departure Time,Arrival Time,From,To,Demand(μ),Demand(σ),Line,P_c,L_c
Trip,,,,,,,,,
1.0,07:00:00,07:46:00,M,A,327.0,64.0,100.0,327.0,300
2.0,07:00:00,07:57:00,H,M,936.0,308.0,800.0,936.0,300
3.0,07:02:00,07:22:00,M,B,461.0,252.0,200.0,461.0,300
4.0,07:04:00,08:03:00,M,J,428.0,10.0,1000.0,428.0,300
5.0,07:06:00,08:13:00,M,F,449.0,124.0,600.0,449.0,300
...,...,...,...,...,...,...,...,...,...
196.0,08:57:00,09:30:00,D,M,384.0,314.0,400.0,384.0,200
197.0,08:58:00,09:26:00,M,D,395.0,366.0,400.0,395.0,200
198.0,08:58:00,09:33:00,M,E,331.0,436.0,500.0,331.0,300


# Solving Fomulation 1

In [3]:
# Sets for model
rolling_stock_types = ['OC', 'OH']
train_services = df.index.to_list()

# Initialise model
formulation1 = gp.Model("Train_Scheduling Formulation 1")

# Constants
costs = {'OC': 260000, 'OH': 210000}
capacity = {'OC': 620, 'OH': 420}
length = {'OC': 100, 'OH': 70}

Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-07


In [4]:
# Decision Variables
N = formulation1.addVars(rolling_stock_types, train_services, vtype=GRB.INTEGER, name='N')

# Objective Function
formulation1.setObjective(quicksum(costs[u] * N[u, c] for u in rolling_stock_types for c in train_services), GRB.MINIMIZE)

# Constraints
# Passenger Demand
demand_constraints = formulation1.addConstrs((quicksum(capacity[u] * N[u, c] for u in rolling_stock_types) >= df.loc[c, 'P_c'] for c in train_services), name="Demand")

# Train Length
length_constraints = formulation1.addConstrs((quicksum(length[u] * N[u, c] for u in rolling_stock_types) <= df.loc[c, 'L_c'] for c in train_services), name="Length")

# Proportionality - ensuring the number of OC and OH units are within 25% of each other
total_OC = quicksum(N['OC', c] for c in train_services)
total_OH = quicksum(N['OH', c] for c in train_services)
formulation1.addConstr(total_OC <= 1.25 * total_OH, "OC_to_OH_proportion")
formulation1.addConstr(total_OH <= 1.25 * total_OC, "OH_to_OC_proportion")

<gurobi.Constr *Awaiting Model Update*>

In [5]:
# Solve the optimisation problem
formulation1.optimize()

results = {(u, c): N[u, c].X for u in rolling_stock_types for c in train_services}

# Convert to DataFrame
unit_df = pd.DataFrame.from_dict(results, orient='index', columns=['Units'])
unit_df.index = pd.MultiIndex.from_tuples(unit_df.index, names=('Rolling Stock Type', 'Service'))
unit_df = unit_df.reset_index()
unit_df = unit_df.pivot(index='Service', columns='Rolling Stock Type', values='Units')

display(unit_df)
print(f"Minimized Total Cost: {formulation1.objVal}")

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 23.1.0 23B2073)

CPU model: Apple M3 Pro
Thread count: 11 physical cores, 11 logical processors, using up to 11 threads

Optimize a model with 402 rows, 400 columns and 1600 nonzeros
Model fingerprint: 0x6ad9b2e0
Variable types: 0 continuous, 400 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+02]
  Objective range  [2e+05, 3e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 1e+03]
Presolve removed 24 rows and 23 columns
Presolve time: 0.00s
Presolved: 378 rows, 377 columns, 1506 nonzeros
Variable types: 0 continuous, 377 integer (0 binary)
Found heuristic solution: objective 9.384000e+07

Root relaxation: objective 7.121878e+07, 282 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 7.1219e+07    0   43 9.3840e+07 7.1219e

Rolling Stock Type,OC,OH
Service,,
1.0,0.0,1.0
2.0,1.0,1.0
3.0,1.0,0.0
4.0,1.0,0.0
5.0,1.0,0.0
...,...,...
196.0,0.0,1.0
197.0,0.0,1.0
198.0,0.0,1.0


Minimized Total Cost: 72940000.0


# Solving Formulation 2

In [6]:
max_oc_units = 300 // length['OC']
max_oh_units = 300 // length['OH']

compositions = []

for oc_units in range(max_oc_units + 1):
    for oh_units in range(max_oh_units + 1):
        total_length = oc_units * length['OC'] + oh_units * length['OH']
        total_capacity = oc_units * capacity['OC'] + oh_units * capacity['OH']
        total_cost = oc_units * costs['OC'] + oh_units * costs['OH']
        
        if total_length <= 300:
            composition_name = f"{oc_units}OC_{oh_units}OH"
            compositions.append((composition_name, total_length, total_capacity, oc_units, oh_units, total_cost))

compositions_df = pd.DataFrame(compositions, columns=['Composition', 'Total Length', 'Total Capacity', 'OC Units', 'OH Units', 'Total Cost'])
compositions_df

,Composition,Total Length,Total Capacity,OC Units,OH Units,Total Cost
0,0OC_0OH,0,0,0,0,0
1,0OC_1OH,70,420,0,1,210000
2,0OC_2OH,140,840,0,2,420000
3,0OC_3OH,210,1260,0,3,630000
4,0OC_4OH,280,1680,0,4,840000
5,1OC_0OH,100,620,1,0,260000
6,1OC_1OH,170,1040,1,1,470000
7,1OC_2OH,240,1460,1,2,680000
8,2OC_0OH,200,1240,2,0,520000
9,2OC_1OH,270,1660,2,1,730000


In [7]:
# remove compositions longer than 200 meters for services on line 400
valid_compositions = {c: compositions_df[compositions_df['Total Length'] <= (200 if df.loc[c, 'Line'] == 400 else 300)]['Composition'].tolist() for c in train_services}

In [8]:
# Initialise model
formulation2= gp.Model('Train_Scheduling, Formulation 2 - Compositions')

# Decision Variables
X = formulation2.addVars(train_services, compositions_df['Composition'], vtype=GRB.INTEGER, name='X')

# Objective Function
formulation2.setObjective(quicksum(X[c, p] * compositions_df.set_index('Composition').loc[p, 'Total Cost'] for c in train_services for p in valid_compositions[c]), GRB.MINIMIZE)

# Constraints
# No Shortages
formulation2.addConstrs((quicksum(X[c, p] * compositions_df.set_index('Composition').loc[p, 'Total Capacity'] for p in valid_compositions[c]) >= df.loc[c, 'P_c'] for c in train_services), name="NoShortages")

# 1 composition per train
formulation2.addConstrs((quicksum(X[c, p] for p in valid_compositions[c]) == 1 for c in train_services), name="OneComposition")

# Solve the optimisation problem
formulation2.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 23.1.0 23B2073)

CPU model: Apple M3 Pro
Thread count: 11 physical cores, 11 logical processors, using up to 11 threads

Optimize a model with 400 rows, 2200 columns and 3960 nonzeros
Model fingerprint: 0x10c1b7ab
Variable types: 0 continuous, 2200 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [2e+05, 8e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+03]
Found heuristic solution: objective 1.215000e+08
Presolve removed 400 rows and 2200 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 11 available processors)

Solution count 2: 7.249e+07 1.215e+08 

Optimal solution found (tolerance 1.00e-04)
Best objective 7.249000000000e+07, best bound 7.249000000000e+07, gap 0.0000%


## Compare the computation times of the different formulations, and mention inthe final report why one formulation performs better than the other. Hint: look at the value of LP relaxations.

comparing the results: we find that formulation 2 performs better as it needs 0 simplex iterations and 

In [9]:
# Create a copy of Formulation 1 for LP relaxation
formulation1_lp = formulation1.copy()


# replace integrality constraints
for var in formulation1_lp.getVars():
    var.vType = GRB.CONTINUOUS
        
# Solve the relaxed problem
formulation1_lp.optimize()


# Create a copy of Formulation 2 for LP relaxation
formulation2_lp = formulation2.copy()

# Relax the binary variables to continuous
for var in formulation2_lp.getVars():
    var.vType = GRB.CONTINUOUS
        
# Solve the relaxed problem
formulation2_lp.optimize()

print("LP Relaxation Objective Value for Formulation 1:", formulation1_lp.objVal)
print("LP Relaxation Objective Value for Formulation 2:", formulation2_lp.objVal)

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 23.1.0 23B2073)

CPU model: Apple M3 Pro
Thread count: 11 physical cores, 11 logical processors, using up to 11 threads

Optimize a model with 402 rows, 400 columns and 1600 nonzeros
Model fingerprint: 0xb8c2420e
Coefficient statistics:
  Matrix range     [1e+00, 6e+02]
  Objective range  [2e+05, 3e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 1e+03]
Presolve time: 0.00s
Presolved: 402 rows, 400 columns, 1600 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   4.268188e+03   0.000000e+00      0s
     275    6.1147590e+07   0.000000e+00   0.000000e+00      0s

Solved in 275 iterations and 0.01 seconds (0.00 work units)
Optimal objective  6.114758996e+07
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 23.1.0 23B2073)

CPU model: Apple M3 Pro
Thread count: 11 physical cores, 11 logical processors, using up to 11 threads

Optimize

In [10]:
# Observe The LP relaxation of formulation 2 gets a much lower Objective value and hence a tighter formulation.
# however formulation 3

Observe The LP relaxation of formulation 2 gets a much lower Objective value and hence a tighter formulation.
however formulation 1 needs fewer iterations. 